In [4]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
import random
import bson

sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define indexes on embedded docs' id
Tickets are embedded inside visitors' docs. Events reference them. Thus, tickets must have a ID to be retrievable.

First I've added a `_id` field to ticket embedded docs, with value _ObjectId_. Then, I defined an index on such a field.

## Performance improvements
Before defining the index:
```python
db.visitors.find({"tickets._id": ObjectId("<OID>")}).explain()
```

winning plan = `COLLSCAN`. `docsExamined = 50`. `totalKeysExamined = 0`. **Query scanned whole collection.**

Then,
```python
db.visitors.create_index({ "tickets._id": 1 })
```

winning plan = `IXSCAN` -> `FETCH`. `indexName = tickets._id_1`. `totalKeysExamined = 1`. `totalDocsExamined = 1`. **Query scanned 1 document.**

FETCH appears because Mongo must read the full document after the index gives the matching key.

## Index update

MongoDB maintains indexes automatically on writes.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [5]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/4_consistent/omero_museum"

In [6]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room start_date ticketIds workshop_title
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donation_state donator_id is_original location_name materials period seller_id size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[limited_events]: _id artist artwork_titles author_ids capacity description end_date room_name start_date theme ticket_ids type
[messages]: _id content customer_id delivery_status length sent_date type uri
[roles]: _id activity_ids birth_date chat_ids curriculum date_start department desk email english_cert gender hometown name password phone_number salary shifts surname type username
[rooms]: _id floor
[suppliers]: _id email iban is_museum is_state name phone_num survey_ids
[visitors]: _id birth_da

## Define indexes on _Embedded_ + _Referenced_ documents

In [7]:
db.visitors.create_index({ "tickets._id": 1 })

'tickets._id_1'

---
## Dump Final Database

In [8]:
%%capture
!mongodump --host localhost:27017 --db omero_museum --out "../backup/5_optimized"